In [1]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define batch size and image dimensions
batch_size = 128
IMG_HEIGHT = 150
IMG_WIDTH = 150

# Load the "cats_vs_dogs" dataset from TensorFlow Datasets (TFDS)
dataset, info = tfds.load('cats_vs_dogs', split='train[:80%]', with_info=True, as_supervised=True)

# Separate the dataset into training and validation sets
total_images = info.splits['train'].num_examples
train_split = total_images * 80 // 100

train_ds = dataset.take(train_split)
val_ds = dataset.skip(train_split)

# Function to preprocess images
def preprocess_image(image, label):
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = tf.cast(image, tf.float32) / 255.0  # Rescale pixel values to [0, 1]
    return image, label

# Apply preprocessing to training and validation datasets
train_ds = train_ds.map(preprocess_image)
val_ds = val_ds.map(preprocess_image)

# Shuffle and batch the datasets
train_ds = train_ds.shuffle(buffer_size=1000).batch(batch_size)
val_ds = val_ds.batch(batch_size)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.1.incompleteGTGCIN/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [2]:
# Define the test split
test_ds, test_info = tfds.load('cats_vs_dogs', split='train[80%:]', with_info=True, as_supervised=True)

# Preprocess the test dataset
test_ds = test_ds.map(preprocess_image)
test_ds = test_ds.batch(batch_size)

In [3]:
# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(train_ds,
                    epochs=15,
                    validation_data=val_ds)

Epoch 1/15
146/146 [==============================] - 497s 3s/step - loss: 0.6988 - accuracy: 0.5962 - val_loss: 0.1559 - val_accuracy: 1.0000
Epoch 2/15
146/146 [==============================] - 495s 3s/step - loss: 0.5440 - accuracy: 0.7240 - val_loss: 0.3266 - val_accuracy: 1.0000
Epoch 3/15
146/146 [==============================] - 491s 3s/step - loss: 0.4467 - accuracy: 0.7902 - val_loss: 0.1050 - val_accuracy: 1.0000
Epoch 4/15
146/146 [==============================] - 497s 3s/step - loss: 0.3709 - accuracy: 0.8331 - val_loss: 0.0141 - val_accuracy: 1.0000
Epoch 5/15
146/146 [==============================] - 498s 3s/step - loss: 0.2990 - accuracy: 0.8730 - val_loss: 0.0518 - val_accuracy: 1.0000
Epoch 6/15
146/146 [==============================] - 497s 3s/step - loss: 0.2182 - accuracy: 0.9108 - val_loss: 0.0307 - val_accuracy: 1.0000
Epoch 7/15
146/146 [==============================] - 494s 3s/step - loss: 0.1385 - accuracy: 0.9488 - val_loss: 0.0021 - val_accuracy: 1.0000

In [ ]:
# Plot training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(15)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_ds)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Function to preprocess a single image
def preprocess_single_image(image):
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = tf.cast(image, tf.float32) / 255.0  # Rescale pixel values to [0, 1]
    return image

# Load a single image from a URL (replace 'image_url' with the URL of the image)
image_url = 'https://example.com/image.jpg'
image_path = tf.keras.utils.get_file('image.jpg', image_url)

# Read the image using TensorFlow
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

# Preprocess the image
preprocessed_image = preprocess_single_image(image)

# Reshape the image to match the expected input shape of the model (batch_size, IMG_HEIGHT, IMG_WIDTH, 3)
preprocessed_image = tf.reshape(preprocessed_image, [1, IMG_HEIGHT, IMG_WIDTH, 3])

# Make predictions using the model
prediction = model.predict(preprocessed_image)

# Display the prediction
if prediction[0][0] > 0.5:
    print("The image is predicted to be a dog.")
else:
    print("The image is predicted to be a cat.")


In [ ]:
# Function to plot images with labels
def plot_images(images, labels):
    plt.figure(figsize=(10, 10))
    for i in range(len(images)):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel('Dog' if labels[i] == 1 else 'Cat')
    plt.show()

# Extract a batch of images and labels from the test dataset
test_images, test_labels = next(iter(test_ds))

# Plot a batch of test images
plot_images(test_images[:25], test_labels[:25])

In [ ]:
# Predict labels for the test dataset
predictions = model.predict(test_ds)

# Convert predictions to binary labels (0 for cat, 1 for dog)
binary_predictions = (predictions > 0.5).astype(int)

# Calculate accuracy
accuracy = np.mean(binary_predictions == test_labels.numpy())

print("Test Accuracy:", accuracy)

In [ ]:
# Plot confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Flatten the true labels and predictions
true_labels = test_labels.numpy().flatten()
predicted_labels = binary_predictions.flatten()

# Calculate confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Check if the challenge is passed
if accuracy >= 0.63:
    print("Congratulations! You passed the challenge.")
else:
    print("Sorry, you did not pass the challenge. Keep trying!")
